In [2]:
import os
import cv2
import time
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from torch.utils.data import DataLoader,Dataset
from torch.utils.data import RandomSampler



In [5]:
import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder


In [6]:
from matplotlib import pyplot as plt

In [21]:
TRAIN_DIR=r"C:\Users\MD DANISH ANWER\species\train\\"
TEST_DIR=r"C:\Users\MD DANISH ANWER\species\test\\"
VALID_DIR=r"C:\Users\MD DANISH ANWER\species\valid\\"

In [22]:
classes=os.listdir(TRAIN_DIR)
print("Total Number of Classes: ",len(classes))

Total Number of Classes:  270


In [24]:
Number_of_training_images=0
Number_of_test_images=0
Number_of_validation_images=0

for class_ in classes:
    Number_of_training_images+=len(os.listdir(TRAIN_DIR+ class_))
    Number_of_test_images+=len(os.listdir(TEST_DIR + class_))
    Number_of_validation_images+=len(os.listdir(VALID_DIR + class_))
print("Number of Train Images: ",Number_of_training_images )
print("Number of validation Images: ",Number_of_validation_images )
print("Number of Test Images: ",Number_of_test_images )

Number of Train Images:  38518
Number of validation Images:  1350
Number of Test Images:  1350


In [35]:
# Creating list of images

Train_imgs=[]
Valid_imgs=[]
Test_imgs=[]

for _class in classes:
    
    for img in os.listdir(TRAIN_DIR+ _class):
        Train_imgs.append(TRAIN_DIR+_class +"\\"+ img)
        
    for img in os.listdir(VALID_DIR+ _class):
        Valid_imgs.append(VALID_DIR +_class +"\\"+ img)
        
    for img in os.listdir(TEST_DIR+ _class):
        Test_imgs.append(TEST_DIR+_class +"\\"+ img)

In [36]:
Train_imgs[0:2]

['C:\\Users\\MD DANISH ANWER\\species\\train\\\\AFRICAN CROWNED CRANE\\001.jpg',
 'C:\\Users\\MD DANISH ANWER\\species\\train\\\\AFRICAN CROWNED CRANE\\002.jpg']

In [37]:
# classes to numerical value

class_to_int={classes[i] : i for i in range(len(classes))}

In [38]:
class_to_int

{'AFRICAN CROWNED CRANE': 0,
 'AFRICAN FIREFINCH': 1,
 'ALBATROSS': 2,
 'ALEXANDRINE PARAKEET': 3,
 'AMERICAN AVOCET': 4,
 'AMERICAN BITTERN': 5,
 'AMERICAN COOT': 6,
 'AMERICAN GOLDFINCH': 7,
 'AMERICAN KESTREL': 8,
 'AMERICAN PIPIT': 9,
 'AMERICAN REDSTART': 10,
 'ANHINGA': 11,
 'ANNAS HUMMINGBIRD': 12,
 'ANTBIRD': 13,
 'ARARIPE MANAKIN': 14,
 'ASIAN CRESTED IBIS': 15,
 'BALD EAGLE': 16,
 'BALI STARLING': 17,
 'BALTIMORE ORIOLE': 18,
 'BANANAQUIT': 19,
 'BANDED BROADBILL': 20,
 'BAR-TAILED GODWIT': 21,
 'BARN OWL': 22,
 'BARN SWALLOW': 23,
 'BARRED PUFFBIRD': 24,
 'BAY-BREASTED WARBLER': 25,
 'BEARDED BARBET': 26,
 'BEARDED REEDLING': 27,
 'BELTED KINGFISHER': 28,
 'BIRD OF PARADISE': 29,
 'BLACK & YELLOW bROADBILL': 30,
 'BLACK FRANCOLIN': 31,
 'BLACK SKIMMER': 32,
 'BLACK SWAN': 33,
 'BLACK TAIL CRAKE': 34,
 'BLACK THROATED BUSHTIT': 35,
 'BLACK THROATED WARBLER': 36,
 'BLACK VULTURE': 37,
 'BLACK-CAPPED CHICKADEE': 38,
 'BLACK-NECKED GREBE': 39,
 'BLACK-THROATED SPARROW': 40,
 'BL

In [43]:
def get_transform():
    return T.Compose([T.ToTensor()])

class BirdDataset(Dataset):
    def __init__(self,imgs_list,class_to_int, transforms=None):
        
        super().__init__()
        self.imgs_list=imgs_list
        self.class_to_int=class_to_int
        self.transforms=transforms

        
    def __getitem__(self,index):
        
        image_path=self.imgs_list[index]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        label = image_path.split("\\")[-2]
        
        label = self.class_to_int[label]
        
        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        return image, label
        
        
        
    def __len__(self):
        return len(self.imgs_list)
        
        
        
        
        
        

In [44]:
train_dataset = BirdDataset(Train_imgs, class_to_int, get_transform())
valid_dataset = BirdDataset(Valid_imgs, class_to_int, get_transform())
test_dataset = BirdDataset(Test_imgs, class_to_int, get_transform())

In [45]:
train_dataset[1]

(tensor([[[0.4784, 0.4824, 0.4941,  ..., 0.4627, 0.4784, 0.4706],
          [0.4784, 0.4824, 0.4941,  ..., 0.4667, 0.4745, 0.4667],
          [0.4824, 0.4863, 0.4863,  ..., 0.4667, 0.4667, 0.4627],
          ...,
          [0.2824, 0.2314, 0.3843,  ..., 0.4510, 0.5412, 0.6510],
          [0.3569, 0.3490, 0.3804,  ..., 0.4784, 0.5451, 0.6314],
          [0.3412, 0.3961, 0.3725,  ..., 0.4078, 0.5059, 0.6039]],
 
         [[0.4196, 0.4235, 0.4235,  ..., 0.3961, 0.4000, 0.3922],
          [0.4196, 0.4235, 0.4235,  ..., 0.4000, 0.3961, 0.3882],
          [0.4235, 0.4275, 0.4275,  ..., 0.4000, 0.3882, 0.3843],
          ...,
          [0.2471, 0.2078, 0.3608,  ..., 0.3804, 0.4627, 0.5686],
          [0.3137, 0.3137, 0.3451,  ..., 0.4078, 0.4627, 0.5451],
          [0.2980, 0.3529, 0.3373,  ..., 0.3373, 0.4235, 0.5176]],
 
         [[0.1569, 0.1608, 0.1725,  ..., 0.1765, 0.2000, 0.1922],
          [0.1569, 0.1608, 0.1725,  ..., 0.1804, 0.1961, 0.1882],
          [0.1686, 0.1725, 0.1725,  ...,